In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os

import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, Sequential
from keras.applications.resnet50 import ResNet50, preprocess_input as resnet50_preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess_input
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.utils.np_utils import to_categorical

from constants import img_size, train_data_dir, dev_data_dir, epochs, batch_size, num_class

Using TensorFlow backend.


In [2]:
def generator(datagen, data_dir, class_mode):
    return datagen.flow_from_directory(
        data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode=class_mode,
        shuffle=False,
        seed=0,
    )

In [3]:
def base_model(model_constructor, name, preprocessor, pooling, verbose):
    datagen = ImageDataGenerator(
        preprocessing_function=preprocessor,
    )
    
    model = model_constructor(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3), pooling=pooling)
    
    if verbose == True:
        model.summary()
        
    return model

In [4]:
def paths(name, pooling):
    paths_to_create = [
        './data/bottleneck_features/' + name,
        './data/bottleneck_features/' + name + '/flat/' + pooling if pooling != None else './data/bottleneck_features/' + name + '/vol/' + pooling,
        './data/bottleneck_features/' + name + '/flat/' + pooling if pooling != None else './data/bottleneck_features/' + name + '/vol/' + pooling,
    ]
    
    for path in paths_to_create:
        if not os.path.exists(path):
            os.makedirs(path)
            
    return paths_to_create[0], paths_to_create[1], paths_to_create[2]

In [9]:
def extractor(model_constructor, name, preprocessor, pooling, verbose = True):
    model = base_model(model_constructor, name, preprocessor, pooling, verbose)
    
    datagen = ImageDataGenerator(preprocessing_function=preprocessor)
    train_generator = generator(datagen, train_data_dir, 'categorical')
    dev_generator = generator(datagen, dev_data_dir, 'categorical')
    
    root_path, train_path, dev_path = paths(name, pooling)
    
    #axillaries
    np.save(root_path + '/filenames.npy', dev_generator.filenames)
    np.save(root_path + '/classes.npy', dev_generator.classes)
    
    #labels
    train_labels = to_categorical(train_generator.classes)
    dev_labels = to_categorical(dev_generator.classes)
    np.save(root_path + '/train_labels.npy', train_labels)
    np.save(root_path + '/dev_labels.npy', dev_labels)
    
    #train features
    bottleneck_features_train = model.predict_generator(train_generator, len(train_generator), verbose=1)
    np.save(train_path + '/bottleneck_features_train.npy', bottleneck_features_train)
    
    #dev features
    bottleneck_features_dev = model.predict_generator(dev_generator, len(dev_generator), verbose=1)
    np.save(dev_path + '/bottleneck_features_dev.npy', bottleneck_features_dev)

In [10]:
extractor(ResNet50, 'res_net', resnet50_preprocess_input, 'avg')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Found 9144 images belonging to 120 classes.
Found 1078 images belonging to 120 classes.
   5/1143 [..............................] - ETA: 50:19

KeyboardInterrupt: 

In [ ]:
extractor(InceptionV3, 'res_net', resnet50_preprocess_input, 'avg')